# Regression analysis and training

In [1]:
import glob
import os
import os.path
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import logging
from sklearn.preprocessing import StandardScaler
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models.annotations import Title
from maweight import model_selection
from config import elastix_params
import pickle
output_notebook()

from config import leg_features_path, loin_features_path, belly_features_path, groin_features_path
from config import shoulder_features_path, body_features_path
from config import xls_path, path_prefix_results

import warnings
warnings.filterwarnings('ignore')

# setting the logging format
FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

Executables being used: /opt/elastix-5.1.0-linux/bin/elastix /opt/elastix-5.1.0-linux/bin/transformix


Loading BokehJS ...

## Constructing the training dataframe

In [2]:
# Reading the extracted features
["leg", "loin", "belly", "groin", "shoulder", "body"]
leg_features= pd.read_csv(leg_features_path)
loin_features= pd.read_csv(loin_features_path)
belly_features= pd.read_csv(belly_features_path)
groin_features= pd.read_csv(groin_features_path)
shoulder_features= pd.read_csv(shoulder_features_path)
body_features= pd.read_csv(body_features_path)

# Determining the ids and positions
leg_features['id']= leg_features['filename'].apply(lambda x: x.split(os.sep)[-1]).apply(lambda x: x[:-7] if x.endswith(".nii.gz") else x[:-4]) 
loin_features['id']= loin_features['filename'].apply(lambda x: x.split(os.sep)[-1]).apply(lambda x: x[:-7] if x.endswith(".nii.gz") else x[:-4])
belly_features['id']= belly_features['filename'].apply(lambda x: x.split(os.sep)[-1]).apply(lambda x: x[:-7] if x.endswith(".nii.gz") else x[:-4]) 
groin_features['id']= groin_features['filename'].apply(lambda x: x.split(os.sep)[-1]).apply(lambda x: x[:-7] if x.endswith(".nii.gz") else x[:-4])
shoulder_features['id']= shoulder_features['filename'].apply(lambda x: x.split(os.sep)[-1]).apply(lambda x: x[:-7] if x.endswith(".nii.gz") else x[:-4]) 
body_features['id']= body_features['filename'].apply(lambda x: x.split(os.sep)[-1]).apply(lambda x: x[:-7] if x.endswith(".nii.gz") else x[:-4])

# Reading the XLS data
xls_data= pd.read_excel(xls_path, engine='openpyxl')

# Extracting the ground truth data
target= pd.DataFrame(data= {'ct_num': xls_data[u'id'].astype(str).str.zfill(3), 
                            'leg': xls_data[u'HAM'].astype(float),
                            'loin': xls_data[u'LOIN'].astype(float),
                            'belly': xls_data[u'BELLY'].astype(float),
                            'groin': xls_data[u'VENTRA_ PART_BELLY'].astype(float), 
                            'shoulder': xls_data[u'SHOULDER'].astype(float),
                            'body': xls_data[u'L_CW'].astype(float)})

target['id']= target['ct_num']
target= target.sort_values('id')
target= target.reset_index()

# Merging the skull features with the ground truth data
leg_target= target[['id', 'leg']]
loin_target= target[['id', 'loin']]
belly_target= target[['id', 'belly']]
groin_target= target[['id', 'groin']]
shoulder_target= target[['id', 'shoulder']]
body_target= target[['id', 'body']]

leg_data= pd.merge(leg_features, leg_target, how='inner', on='id').dropna()
loin_data= pd.merge(loin_features, loin_target, how='inner', on='id').dropna()
belly_data= pd.merge(belly_features, belly_target, how='inner', on='id').dropna()
groin_data= pd.merge(groin_features, groin_target, how='inner', on='id').dropna()
shoulder_data= pd.merge(shoulder_features, shoulder_target, how='inner', on='id').dropna()
body_data= pd.merge(body_features, body_target, how='inner', on='id').dropna()

In [3]:
leg_target= leg_data['leg']
leg_features= leg_data.drop(['filename', 'id', 'leg'], axis='columns')
loin_target= loin_data['loin']
loin_features= loin_data.drop(['filename', 'id', 'loin'], axis='columns')
belly_target= belly_data['belly']
belly_features= belly_data.drop(['filename', 'id', 'belly'], axis='columns')
groin_target= groin_data['groin']
groin_features= groin_data.drop(['filename', 'id', 'groin'], axis='columns')
shoulder_target= shoulder_data['shoulder']
shoulder_features= shoulder_data.drop(['filename', 'id', 'shoulder'], axis='columns')
body_target= body_data['body']
body_features= body_data.drop(['filename', 'id', 'body'], axis='columns')

# encoding the positions
#skull_features['pos']= skull_features['pos'].apply(lambda x: {'a': 0, 'k': 1, 'f': 2}[x])
#head_features['pos']= head_features['pos'].apply(lambda x: {'a': 0, 'k': 1, 'f': 2}[x])

## Model selection with feature selection

In [4]:
masks= np.unique([c.split('-')[1] for c in leg_features.columns if ('mean_mask' not in c and len(c) > 10) and len(c.split('-')[1]) == 4])

In [5]:
masks

array(['p004', 'p029', 'p032', 'p041', 'p048'], dtype='<U4')

In [6]:
results= []
results_no_fs= []

### leg using all features

In [7]:
results.append(model_selection(leg_features, leg_target, dataset='leg', type='all'))

Objective KNNR_Objective:


 18%|███████▏                               | 1472/8000 [03:07<13:50,  7.86it/s]
78it [00:00, 778.55it/s]

iterations: 1472
Number of used features: 24
Used features: ['Unnamed: 0', 'num-p004-leg.nii.gz-0.500000', 'std-p004-leg.nii.gz-0.500000', 'hist-2-p004-leg.nii.gz-0.500000', 'num-p029-leg.nii.gz-0.500000', 'skew-p029-leg.nii.gz-0.500000', 'hist-1-p029-leg.nii.gz-0.500000', 'hist-5-p029-leg.nii.gz-0.500000', 'hist-10-p029-leg.nii.gz-0.500000', 'hist-17-p029-leg.nii.gz-0.500000', 'hist-9-p032-leg.nii.gz-0.500000', 'hist-13-p032-leg.nii.gz-0.500000', 'hist-14-p032-leg.nii.gz-0.500000', 'hist-17-p032-leg.nii.gz-0.500000', 'num-p041-leg.nii.gz-0.500000', 'hist-12-p041-leg.nii.gz-0.500000', 'hist-13-p041-leg.nii.gz-0.500000', 'std-p048-leg.nii.gz-0.500000', 'hist-8-p048-leg.nii.gz-0.500000', 'hist-14-p048-leg.nii.gz-0.500000', 'hist-16-p048-leg.nii.gz-0.500000', 'num-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-7-0.500000-mean_mask'] 
Score: -0.897328883702748


200it [00:00, 777.88it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

1
1 0.90442231799878
Objective LinearRegression_Objective:


...

1
1 0.9895821822928035
Objective PLSRegression_Objective:


 23%|█████████                              | 1857/8000 [04:40<15:26,  6.63it/s]
54it [00:00, 534.99it/s]

iterations: 1857
Number of used features: 17
Used features: ['Unnamed: 0', 'num-p004-leg.nii.gz-0.500000', 'skew-p004-leg.nii.gz-0.500000', 'hist-10-p004-leg.nii.gz-0.500000', 'hist-13-p004-leg.nii.gz-0.500000', 'hist-14-p004-leg.nii.gz-0.500000', 'hist-19-p004-leg.nii.gz-0.500000', 'hist-13-p029-leg.nii.gz-0.500000', 'hist-17-p029-leg.nii.gz-0.500000', 'hist-19-p029-leg.nii.gz-0.500000', 'num-p041-leg.nii.gz-0.500000', 'hist-15-p041-leg.nii.gz-0.500000', 'hist-16-p041-leg.nii.gz-0.500000', 'hist-12-p048-leg.nii.gz-0.500000', 'hist-16-p048-leg.nii.gz-0.500000', 'hist-17-p048-leg.nii.gz-0.500000', 'hist-16-0.500000-mean_mask'] 
Score: -0.9823000280063845


200it [00:00, 534.68it/s]

1
1 0.9842008847890619


### leg using the mean mask features

In [8]:
results.append(model_selection(leg_features[[c for c in leg_features.columns if 'mean_mask' in c or c == 'type']], leg_target, dataset='leg', type='mean_mask'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 542/8000 [00:47<10:55, 11.37it/s]
82it [00:00, 817.24it/s]

iterations: 542
Number of used features: 1
Used features: ['num-0.500000-mean_mask'] 
Score: -0.8927632314382461


200it [00:00, 814.24it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9715156099321257
Objective PLSRegression_Objective:


 12%|████▋                                   | 946/8000 [00:36<04:30, 26.05it/s]
63it [00:00, 628.84it/s]

iterations: 946
Number of used features: 8
Used features: ['num-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.9651899828586591


200it [00:00, 623.46it/s]

1
1 0.9657409357227466


### leg using the features of the individual masks

In [10]:
for m in masks:
    results.append(model_selection(leg_features[[c for c in leg_features.columns if m in c or c == 'type']], leg_target, dataset='leg', type=m))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 14%|█████▌                                 | 1147/8000 [01:48<10:48, 10.57it/s]
78it [00:00, 774.03it/s]

iterations: 1147
Number of used features: 2
Used features: ['num-p004-leg.nii.gz-0.500000', 'mean-p004-leg.nii.gz-0.500000'] 
Score: -0.8980510907322848


200it [00:00, 774.39it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9636086527655654
Objective PLSRegression_Objective:


 13%|█████▏                                 | 1060/8000 [00:51<05:34, 20.74it/s]
66it [00:00, 655.51it/s]

iterations: 1060
Number of used features: 5
Used features: ['num-p048-leg.nii.gz-0.500000', 'hist-9-p048-leg.nii.gz-0.500000', 'hist-15-p048-leg.nii.gz-0.500000', 'hist-18-p048-leg.nii.gz-0.500000', 'hist-19-p048-leg.nii.gz-0.500000'] 
Score: -0.9613693042986711


200it [00:00, 652.49it/s]

1
1 0.9625740776633679


### loin using all features

In [11]:
results.append(model_selection(loin_features, loin_target, dataset='loin', type='all'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 33%|████████████▊                          | 2623/8000 [05:36<11:29,  7.80it/s]
74it [00:00, 733.87it/s]

iterations: 2623
Number of used features: 35
Used features: ['Unnamed: 0', 'num-p004-loin.nii.gz-0.500000', 'hist-0-p004-loin.nii.gz-0.500000', 'hist-1-p004-loin.nii.gz-0.500000', 'hist-3-p004-loin.nii.gz-0.500000', 'num-p029-loin.nii.gz-0.500000', 'std-p029-loin.nii.gz-0.500000', 'hist-2-p029-loin.nii.gz-0.500000', 'hist-3-p029-loin.nii.gz-0.500000', 'hist-14-p029-loin.nii.gz-0.500000', 'hist-17-p029-loin.nii.gz-0.500000', 'hist-19-p029-loin.nii.gz-0.500000', 'num-p032-loin.nii.gz-0.500000', 'sum-p032-loin.nii.gz-0.500000', 'std-p032-loin.nii.gz-0.500000', 'hist-2-p032-loin.nii.gz-0.500000', 'hist-4-p032-loin.nii.gz-0.500000', 'hist-16-p032-loin.nii.gz-0.500000', 'num-p041-loin.nii.gz-0.500000', 'hist-0-p041-loin.nii.gz-0.500000', 'hist-4-p041-loin.nii.gz-0.500000', 'hist-5-p041-loin.nii.gz-0.500000', 'hist-12-p041-loin.nii.gz-0.500000', 'hist-18-p041-loin.nii.gz-0.500000', 'num-p048-loin.nii.gz-0.500000', 'mean-p048-loin.nii.gz-0.500000', 'hist-0-p048-loin.nii.gz-0.500000', 'hist-5-p

200it [00:00, 737.92it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9668355485249978
Objective PLSRegression_Objective:


 22%|████████▋                              | 1770/8000 [04:33<16:01,  6.48it/s]
46it [00:00, 459.91it/s]

iterations: 1770
Number of used features: 14
Used features: ['hist-0-p004-loin.nii.gz-0.500000', 'hist-6-p004-loin.nii.gz-0.500000', 'hist-8-p004-loin.nii.gz-0.500000', 'num-p029-loin.nii.gz-0.500000', 'hist-8-p029-loin.nii.gz-0.500000', 'hist-13-p029-loin.nii.gz-0.500000', 'hist-10-p032-loin.nii.gz-0.500000', 'hist-12-p032-loin.nii.gz-0.500000', 'hist-14-p032-loin.nii.gz-0.500000', 'hist-10-p041-loin.nii.gz-0.500000', 'hist-14-p041-loin.nii.gz-0.500000', 'hist-11-p048-loin.nii.gz-0.500000', 'hist-12-p048-loin.nii.gz-0.500000', 'hist-19-0.500000-mean_mask'] 
Score: -0.9540621625444551


200it [00:00, 457.09it/s]

1
1 0.9616842973721387


### loin using the mean mask features

In [12]:
results.append(model_selection(loin_features[[c for c in loin_features.columns if 'mean_mask' in c or c == 'type']], loin_target, dataset='loin', type='mean_mask'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 527/8000 [00:46<10:58, 11.34it/s]
80it [00:00, 796.09it/s]

iterations: 527
Number of used features: 2
Used features: ['num-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.8543841302973586


200it [00:00, 795.09it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9344352716032137
Objective PLSRegression_Objective:


  8%|███▍                                    | 679/8000 [00:31<05:44, 21.23it/s]
69it [00:00, 686.56it/s]

iterations: 679
Number of used features: 5
Used features: ['num-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.930582813536284


200it [00:00, 690.10it/s]

1
1 0.9323229324118252


### loin using the features of the individual masks

In [13]:
for m in masks:
    results.append(model_selection(loin_features[[c for c in loin_features.columns if m in c or c == 'type']], loin_target, dataset='loin', type=m))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▉                                     | 588/8000 [00:53<11:08, 11.08it/s]
81it [00:00, 801.14it/s]

iterations: 588
Number of used features: 2
Used features: ['num-p004-loin.nii.gz-0.500000', 'hist-19-p004-loin.nii.gz-0.500000'] 
Score: -0.8279084475373826


200it [00:00, 799.86it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9279206134430613
Objective PLSRegression_Objective:


  7%|██▉                                     | 591/8000 [00:26<05:29, 22.50it/s]
70it [00:00, 695.77it/s]

iterations: 591
Number of used features: 5
Used features: ['num-p048-loin.nii.gz-0.500000', 'sum-p048-loin.nii.gz-0.500000', 'mean-p048-loin.nii.gz-0.500000', 'skew-p048-loin.nii.gz-0.500000', 'hist-4-p048-loin.nii.gz-0.500000'] 
Score: -0.9296573318681052


200it [00:00, 695.16it/s]

1
1 0.9301049953934944


### belly using all features

In [14]:
results.append(model_selection(belly_features, belly_target, dataset='belly', type='all'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 28%|██████████▉                            | 2254/8000 [04:56<12:34,  7.61it/s]
78it [00:00, 771.10it/s]

iterations: 2254
Number of used features: 32
Used features: ['num-p004-belly.nii.gz-0.500000', 'hist-4-p004-belly.nii.gz-0.500000', 'hist-8-p004-belly.nii.gz-0.500000', 'num-p029-belly.nii.gz-0.500000', 'hist-2-p029-belly.nii.gz-0.500000', 'hist-9-p029-belly.nii.gz-0.500000', 'hist-12-p029-belly.nii.gz-0.500000', 'hist-13-p029-belly.nii.gz-0.500000', 'num-p032-belly.nii.gz-0.500000', 'kurt-p032-belly.nii.gz-0.500000', 'hist-0-p032-belly.nii.gz-0.500000', 'hist-4-p032-belly.nii.gz-0.500000', 'hist-9-p032-belly.nii.gz-0.500000', 'num-p041-belly.nii.gz-0.500000', 'kurt-p041-belly.nii.gz-0.500000', 'hist-2-p041-belly.nii.gz-0.500000', 'hist-4-p041-belly.nii.gz-0.500000', 'hist-9-p041-belly.nii.gz-0.500000', 'hist-13-p041-belly.nii.gz-0.500000', 'sum-p048-belly.nii.gz-0.500000', 'std-p048-belly.nii.gz-0.500000', 'hist-0-p048-belly.nii.gz-0.500000', 'hist-2-p048-belly.nii.gz-0.500000', 'hist-6-p048-belly.nii.gz-0.500000', 'hist-8-p048-belly.nii.gz-0.500000', 'hist-16-p048-belly.nii.gz-0.5000

200it [00:00, 769.02it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9963583212998265
Objective PLSRegression_Objective:


 33%|████████████▉                          | 2647/8000 [07:26<15:02,  5.93it/s]
45it [00:00, 442.85it/s]

iterations: 2647
Number of used features: 21
Used features: ['hist-3-p004-belly.nii.gz-0.500000', 'hist-14-p004-belly.nii.gz-0.500000', 'hist-1-p029-belly.nii.gz-0.500000', 'hist-2-p029-belly.nii.gz-0.500000', 'hist-17-p029-belly.nii.gz-0.500000', 'hist-19-p029-belly.nii.gz-0.500000', 'std-p032-belly.nii.gz-0.500000', 'hist-12-p032-belly.nii.gz-0.500000', 'hist-13-p032-belly.nii.gz-0.500000', 'skew-p041-belly.nii.gz-0.500000', 'hist-0-p041-belly.nii.gz-0.500000', 'hist-14-p041-belly.nii.gz-0.500000', 'hist-18-p041-belly.nii.gz-0.500000', 'hist-3-p048-belly.nii.gz-0.500000', 'hist-12-p048-belly.nii.gz-0.500000', 'hist-13-p048-belly.nii.gz-0.500000', 'sum-0.500000-mean_mask', 'std-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.9779134877973925


200it [00:00, 445.26it/s]

1
1 0.978600437436421


### belly using the mean mask features

In [15]:
results.append(model_selection(belly_features[[c for c in belly_features.columns if 'mean_mask' in c or c == 'type']], belly_target, dataset='belly', type='mean_mask'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▊                                     | 552/8000 [00:50<11:19, 10.97it/s]
79it [00:00, 784.55it/s]

iterations: 552
Number of used features: 5
Used features: ['num-0.500000-mean_mask', 'std-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.844001745041048


200it [00:00, 777.18it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9057930375876103
Objective PLSRegression_Objective:


 10%|███▉                                    | 786/8000 [00:38<05:54, 20.37it/s]
74it [00:00, 734.14it/s]

iterations: 786
Number of used features: 3
Used features: ['num-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-10-0.500000-mean_mask'] 
Score: -0.9042673652689205


200it [00:00, 730.49it/s]

1
1 0.9056000222671949


### belly using the features of the individual masks

In [17]:
for m in masks:
    results.append(model_selection(belly_features[[c for c in belly_features.columns if m in c or c == 'type']], belly_target, dataset='belly', type=m))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 10%|███▊                                    | 761/8000 [01:12<11:29, 10.50it/s]
81it [00:00, 802.19it/s]

iterations: 761
Number of used features: 5
Used features: ['num-p004-belly.nii.gz-0.500000', 'sum-p004-belly.nii.gz-0.500000', 'hist-2-p004-belly.nii.gz-0.500000', 'hist-8-p004-belly.nii.gz-0.500000', 'hist-12-p004-belly.nii.gz-0.500000'] 
Score: -0.8214747805718592


200it [00:00, 796.53it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8851529534912131
Objective PLSRegression_Objective:


 16%|██████▏                                | 1271/8000 [00:55<04:53, 22.93it/s]
65it [00:00, 648.27it/s]

iterations: 1271
Number of used features: 8
Used features: ['sum-p048-belly.nii.gz-0.500000', 'skew-p048-belly.nii.gz-0.500000', 'kurt-p048-belly.nii.gz-0.500000', 'hist-1-p048-belly.nii.gz-0.500000', 'hist-2-p048-belly.nii.gz-0.500000', 'hist-10-p048-belly.nii.gz-0.500000', 'hist-12-p048-belly.nii.gz-0.500000', 'hist-14-p048-belly.nii.gz-0.500000'] 
Score: -0.8731897933228021


200it [00:00, 647.60it/s]

1
1 0.8735519683030296


### groin using all features

In [18]:
results.append(model_selection(groin_features, groin_target, dataset='groin', type='all'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 21%|███████▉                               | 1641/8000 [03:33<13:47,  7.68it/s]
75it [00:00, 742.54it/s]

iterations: 1641
Number of used features: 25
Used features: ['std-p004-groin.nii.gz-0.500000', 'hist-12-p004-groin.nii.gz-0.500000', 'hist-16-p004-groin.nii.gz-0.500000', 'num-p029-groin.nii.gz-0.500000', 'sum-p029-groin.nii.gz-0.500000', 'hist-9-p029-groin.nii.gz-0.500000', 'hist-10-p029-groin.nii.gz-0.500000', 'hist-15-p029-groin.nii.gz-0.500000', 'hist-16-p029-groin.nii.gz-0.500000', 'num-p032-groin.nii.gz-0.500000', 'sum-p032-groin.nii.gz-0.500000', 'hist-6-p032-groin.nii.gz-0.500000', 'hist-9-p032-groin.nii.gz-0.500000', 'hist-16-p032-groin.nii.gz-0.500000', 'sum-p041-groin.nii.gz-0.500000', 'hist-19-p041-groin.nii.gz-0.500000', 'skew-p048-groin.nii.gz-0.500000', 'kurt-p048-groin.nii.gz-0.500000', 'hist-1-p048-groin.nii.gz-0.500000', 'hist-3-p048-groin.nii.gz-0.500000', 'hist-8-p048-groin.nii.gz-0.500000', 'hist-15-p048-groin.nii.gz-0.500000', 'hist-17-p048-groin.nii.gz-0.500000', 'hist-19-p048-groin.nii.gz-0.500000', 'hist-19-0.500000-mean_mask'] 
Score: -0.676971772867668


200it [00:00, 747.36it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9543430650277877
Objective PLSRegression_Objective:


 20%|███████▉                               | 1629/8000 [03:44<14:37,  7.26it/s]
78it [00:00, 779.26it/s]

iterations: 1629
Number of used features: 25
Used features: ['hist-2-p004-groin.nii.gz-0.500000', 'hist-5-p004-groin.nii.gz-0.500000', 'hist-8-p004-groin.nii.gz-0.500000', 'hist-12-p004-groin.nii.gz-0.500000', 'hist-18-p004-groin.nii.gz-0.500000', 'hist-1-p029-groin.nii.gz-0.500000', 'hist-2-p029-groin.nii.gz-0.500000', 'hist-5-p029-groin.nii.gz-0.500000', 'hist-8-p029-groin.nii.gz-0.500000', 'hist-12-p029-groin.nii.gz-0.500000', 'hist-5-p032-groin.nii.gz-0.500000', 'hist-19-p032-groin.nii.gz-0.500000', 'hist-5-p041-groin.nii.gz-0.500000', 'hist-19-p041-groin.nii.gz-0.500000', 'std-p048-groin.nii.gz-0.500000', 'kurt-p048-groin.nii.gz-0.500000', 'hist-5-p048-groin.nii.gz-0.500000', 'hist-11-p048-groin.nii.gz-0.500000', 'hist-14-p048-groin.nii.gz-0.500000', 'hist-16-p048-groin.nii.gz-0.500000', 'hist-17-p048-groin.nii.gz-0.500000', 'hist-19-p048-groin.nii.gz-0.500000', 'sum-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.7400742515353941


200it [00:00, 777.43it/s]

1
1 0.7403908792829689


### groin using the mean mask features

In [19]:
results.append(model_selection(groin_features[[c for c in groin_features.columns if 'mean_mask' in c or c == 'type']], groin_target, dataset='groin', type='mean_mask'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 11%|████▌                                   | 911/8000 [01:34<12:12,  9.68it/s]
79it [00:00, 784.69it/s]

iterations: 911
Number of used features: 8
Used features: ['hist-1-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.5967325717137868


200it [00:00, 785.89it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.7042038823578866
Objective PLSRegression_Objective:


 10%|███▊                                    | 773/8000 [00:44<07:00, 17.18it/s]
66it [00:00, 653.99it/s]

iterations: 773
Number of used features: 7
Used features: ['num-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.7085013357182505


200it [00:00, 649.85it/s]

1
1 0.7097093204055982


### groin using the features of the individual masks

In [20]:
for m in masks:
    results.append(model_selection(groin_features[[c for c in groin_features.columns if m in c or c == 'type']], groin_target, dataset='groin', type=m))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 17%|██████▌                                | 1346/8000 [02:20<11:35,  9.57it/s]
77it [00:00, 767.70it/s]

iterations: 1346
Number of used features: 11
Used features: ['num-p004-groin.nii.gz-0.500000', 'std-p004-groin.nii.gz-0.500000', 'skew-p004-groin.nii.gz-0.500000', 'kurt-p004-groin.nii.gz-0.500000', 'hist-4-p004-groin.nii.gz-0.500000', 'hist-10-p004-groin.nii.gz-0.500000', 'hist-11-p004-groin.nii.gz-0.500000', 'hist-12-p004-groin.nii.gz-0.500000', 'hist-13-p004-groin.nii.gz-0.500000', 'hist-14-p004-groin.nii.gz-0.500000', 'hist-18-p004-groin.nii.gz-0.500000'] 
Score: -0.4933352596621958


200it [00:00, 765.01it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.7158666297731167
Objective PLSRegression_Objective:


 10%|████                                    | 803/8000 [00:53<08:00, 14.97it/s]
63it [00:00, 625.14it/s]

iterations: 803
Number of used features: 8
Used features: ['num-p048-groin.nii.gz-0.500000', 'sum-p048-groin.nii.gz-0.500000', 'hist-0-p048-groin.nii.gz-0.500000', 'hist-8-p048-groin.nii.gz-0.500000', 'hist-10-p048-groin.nii.gz-0.500000', 'hist-11-p048-groin.nii.gz-0.500000', 'hist-17-p048-groin.nii.gz-0.500000', 'hist-19-p048-groin.nii.gz-0.500000'] 
Score: -0.6902972731359991


200it [00:00, 618.66it/s]

1
1 0.6998592789673966


### shoulder using all features

In [21]:
results.append(model_selection(shoulder_features, shoulder_target, dataset='shoulder', type='all'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 11%|████▎                                   | 874/8000 [01:46<14:30,  8.19it/s]
78it [00:00, 769.78it/s]

iterations: 874
Number of used features: 12
Used features: ['mean-p004-shoulder.nii.gz-0.500000', 'std-p004-shoulder.nii.gz-0.500000', 'hist-0-p004-shoulder.nii.gz-0.500000', 'num-p029-shoulder.nii.gz-0.500000', 'hist-2-p029-shoulder.nii.gz-0.500000', 'hist-8-p029-shoulder.nii.gz-0.500000', 'hist-11-p029-shoulder.nii.gz-0.500000', 'hist-14-p029-shoulder.nii.gz-0.500000', 'hist-14-p032-shoulder.nii.gz-0.500000', 'hist-3-p048-shoulder.nii.gz-0.500000', 'hist-8-p048-shoulder.nii.gz-0.500000', 'kurt-0.500000-mean_mask'] 
Score: -0.8314954513114325


200it [00:00, 768.71it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9893349210678095
Objective PLSRegression_Objective:


 25%|█████████▋                             | 1982/8000 [04:41<14:13,  7.05it/s]
70it [00:00, 693.35it/s]

iterations: 1982
Number of used features: 32
Used features: ['sum-p004-shoulder.nii.gz-0.500000', 'mean-p004-shoulder.nii.gz-0.500000', 'hist-1-p004-shoulder.nii.gz-0.500000', 'num-p029-shoulder.nii.gz-0.500000', 'std-p029-shoulder.nii.gz-0.500000', 'hist-12-p029-shoulder.nii.gz-0.500000', 'hist-14-p029-shoulder.nii.gz-0.500000', 'hist-13-p032-shoulder.nii.gz-0.500000', 'hist-14-p032-shoulder.nii.gz-0.500000', 'hist-16-p032-shoulder.nii.gz-0.500000', 'num-p041-shoulder.nii.gz-0.500000', 'hist-11-p041-shoulder.nii.gz-0.500000', 'hist-13-p041-shoulder.nii.gz-0.500000', 'hist-14-p041-shoulder.nii.gz-0.500000', 'hist-15-p041-shoulder.nii.gz-0.500000', 'sum-p048-shoulder.nii.gz-0.500000', 'mean-p048-shoulder.nii.gz-0.500000', 'std-p048-shoulder.nii.gz-0.500000', 'kurt-p048-shoulder.nii.gz-0.500000', 'hist-0-p048-shoulder.nii.gz-0.500000', 'hist-8-p048-shoulder.nii.gz-0.500000', 'hist-11-p048-shoulder.nii.gz-0.500000', 'hist-12-p048-shoulder.nii.gz-0.500000', 'hist-13-p048-shoulder.nii.gz-0.

200it [00:00, 690.02it/s]

1
1 0.9531471935681589


### shoulder using the mean mask features

In [22]:
results.append(model_selection(shoulder_features[[c for c in shoulder_features.columns if 'mean_mask' in c or c == 'type']], shoulder_target, dataset='shoulder', type='mean_mask'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 11%|████▍                                   | 891/8000 [01:30<12:02,  9.83it/s]
77it [00:00, 767.71it/s]

iterations: 891
Number of used features: 5
Used features: ['num-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask'] 
Score: -0.784241590513167


200it [00:00, 763.52it/s]
  0%|                                          | 1/8000 [00:00<13:20,  9.99it/s]

...

1
1 0.9012433932071573
Objective PLSRegression_Objective:


 12%|████▉                                   | 987/8000 [01:01<07:14, 16.16it/s]
81it [00:00, 809.15it/s]

iterations: 987
Number of used features: 8
Used features: ['num-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask'] 
Score: -0.9164024723597688


200it [00:00, 807.32it/s]

1
1 0.9177386010934734


### shoulder using the features of the individual masks

In [23]:
for m in masks:
    results.append(model_selection(shoulder_features[[c for c in shoulder_features.columns if m in c or c == 'type']], shoulder_target, dataset='shoulder', type=m))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  9%|███▊                                    | 757/8000 [01:13<11:44, 10.28it/s]
79it [00:00, 780.72it/s]

iterations: 757
Number of used features: 7
Used features: ['num-p004-shoulder.nii.gz-0.500000', 'mean-p004-shoulder.nii.gz-0.500000', 'std-p004-shoulder.nii.gz-0.500000', 'hist-9-p004-shoulder.nii.gz-0.500000', 'hist-10-p004-shoulder.nii.gz-0.500000', 'hist-12-p004-shoulder.nii.gz-0.500000', 'hist-15-p004-shoulder.nii.gz-0.500000'] 
Score: -0.7917122458692971


200it [00:00, 781.37it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9242125471115369
Objective PLSRegression_Objective:


  9%|███▋                                    | 733/8000 [00:40<06:43, 18.01it/s]
65it [00:00, 647.81it/s]

iterations: 733
Number of used features: 9
Used features: ['num-p048-shoulder.nii.gz-0.500000', 'skew-p048-shoulder.nii.gz-0.500000', 'kurt-p048-shoulder.nii.gz-0.500000', 'hist-0-p048-shoulder.nii.gz-0.500000', 'hist-4-p048-shoulder.nii.gz-0.500000', 'hist-5-p048-shoulder.nii.gz-0.500000', 'hist-11-p048-shoulder.nii.gz-0.500000', 'hist-13-p048-shoulder.nii.gz-0.500000', 'hist-18-p048-shoulder.nii.gz-0.500000'] 
Score: -0.8997375865318269


200it [00:00, 641.54it/s]

1
1 0.9059220405083036


### body using all features

In [24]:
results.append(model_selection(body_features, body_target, dataset='body', type='all'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 24%|█████████▏                             | 1896/8000 [04:23<14:08,  7.19it/s]
74it [00:00, 739.29it/s]

iterations: 1896
Number of used features: 46
Used features: ['num-p004-body.nii.gz-0.500000', 'hist-1-p004-body.nii.gz-0.500000', 'hist-2-p004-body.nii.gz-0.500000', 'hist-3-p004-body.nii.gz-0.500000', 'hist-5-p004-body.nii.gz-0.500000', 'hist-6-p004-body.nii.gz-0.500000', 'hist-7-p004-body.nii.gz-0.500000', 'hist-10-p004-body.nii.gz-0.500000', 'hist-11-p004-body.nii.gz-0.500000', 'hist-31-p004-body.nii.gz-0.500000', 'hist-39-p004-body.nii.gz-0.500000', 'hist-41-p004-body.nii.gz-0.500000', 'hist-42-p004-body.nii.gz-0.500000', 'num-p029-body.nii.gz-0.500000', 'hist-3-p029-body.nii.gz-0.500000', 'hist-39-p029-body.nii.gz-0.500000', 'hist-40-p029-body.nii.gz-0.500000', 'hist-41-p029-body.nii.gz-0.500000', 'hist-43-p029-body.nii.gz-0.500000', 'num-p032-body.nii.gz-0.500000', 'hist-4-p032-body.nii.gz-0.500000', 'hist-5-p032-body.nii.gz-0.500000', 'hist-30-p032-body.nii.gz-0.500000', 'hist-39-p032-body.nii.gz-0.500000', 'hist-40-p032-body.nii.gz-0.500000', 'hist-43-p032-body.nii.gz-0.500000'

200it [00:00, 741.11it/s]
  0%|                                          | 1/8000 [00:00<13:29,  9.88it/s]

...

1
1 0.9894895267396497
Objective PLSRegression_Objective:


 48%|██████████████████▋                    | 3823/8000 [11:50<12:55,  5.38it/s]
59it [00:00, 582.62it/s]

iterations: 3823
Number of used features: 56
Used features: ['num-p004-body.nii.gz-0.500000', 'hist-4-p004-body.nii.gz-0.500000', 'hist-8-p004-body.nii.gz-0.500000', 'hist-14-p004-body.nii.gz-0.500000', 'hist-17-p004-body.nii.gz-0.500000', 'hist-25-p004-body.nii.gz-0.500000', 'hist-27-p004-body.nii.gz-0.500000', 'hist-34-p004-body.nii.gz-0.500000', 'hist-43-p004-body.nii.gz-0.500000', 'num-p029-body.nii.gz-0.500000', 'mean-p029-body.nii.gz-0.500000', 'std-p029-body.nii.gz-0.500000', 'hist-3-p029-body.nii.gz-0.500000', 'hist-4-p029-body.nii.gz-0.500000', 'hist-8-p029-body.nii.gz-0.500000', 'hist-17-p029-body.nii.gz-0.500000', 'hist-31-p029-body.nii.gz-0.500000', 'hist-32-p029-body.nii.gz-0.500000', 'hist-33-p029-body.nii.gz-0.500000', 'hist-39-p029-body.nii.gz-0.500000', 'num-p032-body.nii.gz-0.500000', 'hist-5-p032-body.nii.gz-0.500000', 'hist-8-p032-body.nii.gz-0.500000', 'hist-17-p032-body.nii.gz-0.500000', 'hist-18-p032-body.nii.gz-0.500000', 'hist-19-p032-body.nii.gz-0.500000', 'hi

200it [00:00, 581.73it/s]

1
1 0.9894312944519109


### body using the mean mask features

In [25]:
results.append(model_selection(body_features[[c for c in body_features.columns if 'mean_mask' in c or c == 'type']], body_target, dataset='body', type='mean_mask'))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 13%|████▉                                  | 1025/8000 [02:10<14:48,  7.85it/s]
68it [00:00, 672.65it/s]

iterations: 1025
Number of used features: 9
Used features: ['num-0.500000-mean_mask', 'std-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-25-0.500000-mean_mask', 'hist-42-0.500000-mean_mask', 'hist-43-0.500000-mean_mask'] 
Score: -0.8988918609413632


200it [00:00, 728.90it/s]
  0%|                                          | 1/8000 [00:00<13:20,  9.99it/s]

...

1
1 0.9852844636156
Objective PLSRegression_Objective:


 13%|█████                                  | 1036/8000 [02:28<16:39,  6.97it/s]
59it [00:00, 588.60it/s]

iterations: 1036
Number of used features: 15
Used features: ['num-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-23-0.500000-mean_mask', 'hist-25-0.500000-mean_mask', 'hist-40-0.500000-mean_mask'] 
Score: -0.9846058535458381


200it [00:00, 595.68it/s]

1
1 0.9848702965212244


### body using the features of the individual masks

In [26]:
for m in masks:
    results.append(model_selection(body_features[[c for c in body_features.columns if m in c or c == 'type']], body_target, dataset='body', type=m))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 14%|█████▎                                 | 1089/8000 [02:15<14:18,  8.05it/s]
79it [00:00, 788.95it/s]

iterations: 1089
Number of used features: 10
Used features: ['num-p004-body.nii.gz-0.500000', 'skew-p004-body.nii.gz-0.500000', 'kurt-p004-body.nii.gz-0.500000', 'hist-1-p004-body.nii.gz-0.500000', 'hist-3-p004-body.nii.gz-0.500000', 'hist-4-p004-body.nii.gz-0.500000', 'hist-5-p004-body.nii.gz-0.500000', 'hist-25-p004-body.nii.gz-0.500000', 'hist-42-p004-body.nii.gz-0.500000', 'hist-43-p004-body.nii.gz-0.500000'] 
Score: -0.880703677596003


200it [00:00, 793.38it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.977707885213198
Objective PLSRegression_Objective:


 14%|█████▌                                 | 1129/8000 [02:39<16:13,  7.06it/s]
62it [00:00, 615.43it/s]

iterations: 1129
Number of used features: 10
Used features: ['num-p048-body.nii.gz-0.500000', 'sum-p048-body.nii.gz-0.500000', 'hist-6-p048-body.nii.gz-0.500000', 'hist-8-p048-body.nii.gz-0.500000', 'hist-17-p048-body.nii.gz-0.500000', 'hist-18-p048-body.nii.gz-0.500000', 'hist-21-p048-body.nii.gz-0.500000', 'hist-24-p048-body.nii.gz-0.500000', 'hist-30-p048-body.nii.gz-0.500000', 'hist-43-p048-body.nii.gz-0.500000'] 
Score: -0.9806057894926818


200it [00:00, 613.90it/s]

1
1 0.9804053332533392


### Saving the results

In [27]:
results= pd.concat(results)
results.to_csv(os.path.join(path_prefix_results,'results.csv'), index=False)
pickle.dump(results, open(os.path.join(path_prefix_results,'results.pickle'), 'wb'))

## Without feature selection

### leg with all features

In [28]:
results_no_fs.append(model_selection(leg_features, leg_target, dataset='leg', type='all', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 529/8000 [00:20<04:56, 25.21it/s]
67it [00:00, 661.01it/s]

iterations: 529
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-leg.nii.gz-0.500000', 'sum-p004-leg.nii.gz-0.500000', 'mean-p004-leg.nii.gz-0.500000', 'std-p004-leg.nii.gz-0.500000', 'skew-p004-leg.nii.gz-0.500000', 'kurt-p004-leg.nii.gz-0.500000', 'hist-0-p004-leg.nii.gz-0.500000', 'hist-1-p004-leg.nii.gz-0.500000', 'hist-2-p004-leg.nii.gz-0.500000', 'hist-3-p004-leg.nii.gz-0.500000', 'hist-4-p004-leg.nii.gz-0.500000', 'hist-5-p004-leg.nii.gz-0.500000', 'hist-6-p004-leg.nii.gz-0.500000', 'hist-7-p004-leg.nii.gz-0.500000', 'hist-8-p004-leg.nii.gz-0.500000', 'hist-9-p004-leg.nii.gz-0.500000', 'hist-10-p004-leg.nii.gz-0.500000', 'hist-11-p004-leg.nii.gz-0.500000', 'hist-12-p004-leg.nii.gz-0.500000', 'hist-13-p004-leg.nii.gz-0.500000', 'hist-14-p004-leg.nii.gz-0.500000', 'hist-15-p004-leg.nii.gz-0.500000', 'hist-16-p004-leg.nii.gz-0.500000', 'hist-17-p004-leg.nii.gz-0.500000', 'hist-18-p004-leg.nii.gz-0.500000', 'hist-19-p004-leg.nii.gz-0.500000', 'num-p029-leg.nii.gz

200it [00:00, 657.51it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9447086724046208
Objective PLSRegression_Objective:


  6%|██▍                                    | 504/8000 [00:02<00:31, 236.79it/s]
49it [00:00, 483.24it/s]

iterations: 504
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-leg.nii.gz-0.500000', 'sum-p004-leg.nii.gz-0.500000', 'mean-p004-leg.nii.gz-0.500000', 'std-p004-leg.nii.gz-0.500000', 'skew-p004-leg.nii.gz-0.500000', 'kurt-p004-leg.nii.gz-0.500000', 'hist-0-p004-leg.nii.gz-0.500000', 'hist-1-p004-leg.nii.gz-0.500000', 'hist-2-p004-leg.nii.gz-0.500000', 'hist-3-p004-leg.nii.gz-0.500000', 'hist-4-p004-leg.nii.gz-0.500000', 'hist-5-p004-leg.nii.gz-0.500000', 'hist-6-p004-leg.nii.gz-0.500000', 'hist-7-p004-leg.nii.gz-0.500000', 'hist-8-p004-leg.nii.gz-0.500000', 'hist-9-p004-leg.nii.gz-0.500000', 'hist-10-p004-leg.nii.gz-0.500000', 'hist-11-p004-leg.nii.gz-0.500000', 'hist-12-p004-leg.nii.gz-0.500000', 'hist-13-p004-leg.nii.gz-0.500000', 'hist-14-p004-leg.nii.gz-0.500000', 'hist-15-p004-leg.nii.gz-0.500000', 'hist-16-p004-leg.nii.gz-0.500000', 'hist-17-p004-leg.nii.gz-0.500000', 'hist-18-p004-leg.nii.gz-0.500000', 'hist-19-p004-leg.nii.gz-0.500000', 'num-p029-leg.nii.gz

200it [00:00, 494.17it/s]

1
1 0.9480459174574987


### leg mean mask

In [29]:
results_no_fs.append(model_selection(leg_features[[c for c in leg_features.columns if 'mean_mask' in c or c == 'type']], leg_target, dataset='leg', type='mean_mask', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 529/8000 [00:15<03:37, 34.37it/s]
75it [00:00, 745.00it/s]

iterations: 529
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.745595010259229


200it [00:00, 746.72it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9523854074266522
Objective PLSRegression_Objective:


  6%|██▎                                   | 499/8000 [00:00<00:06, 1072.55it/s]
63it [00:00, 627.93it/s]

iterations: 499
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.9394563548498512


200it [00:00, 622.90it/s]

1
1 0.9457529756602804


### leg individual masks

In [30]:
for m in masks:
    results_no_fs.append(model_selection(leg_features[[c for c in leg_features.columns if m in c or c == 'type']], leg_target, dataset='leg', type=m, disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 529/8000 [00:15<03:32, 35.20it/s]
78it [00:00, 775.01it/s]

iterations: 529
Number of used features: 26
Used features: ['num-p004-leg.nii.gz-0.500000', 'sum-p004-leg.nii.gz-0.500000', 'mean-p004-leg.nii.gz-0.500000', 'std-p004-leg.nii.gz-0.500000', 'skew-p004-leg.nii.gz-0.500000', 'kurt-p004-leg.nii.gz-0.500000', 'hist-0-p004-leg.nii.gz-0.500000', 'hist-1-p004-leg.nii.gz-0.500000', 'hist-2-p004-leg.nii.gz-0.500000', 'hist-3-p004-leg.nii.gz-0.500000', 'hist-4-p004-leg.nii.gz-0.500000', 'hist-5-p004-leg.nii.gz-0.500000', 'hist-6-p004-leg.nii.gz-0.500000', 'hist-7-p004-leg.nii.gz-0.500000', 'hist-8-p004-leg.nii.gz-0.500000', 'hist-9-p004-leg.nii.gz-0.500000', 'hist-10-p004-leg.nii.gz-0.500000', 'hist-11-p004-leg.nii.gz-0.500000', 'hist-12-p004-leg.nii.gz-0.500000', 'hist-13-p004-leg.nii.gz-0.500000', 'hist-14-p004-leg.nii.gz-0.500000', 'hist-15-p004-leg.nii.gz-0.500000', 'hist-16-p004-leg.nii.gz-0.500000', 'hist-17-p004-leg.nii.gz-0.500000', 'hist-18-p004-leg.nii.gz-0.500000', 'hist-19-p004-leg.nii.gz-0.500000'] 
Score: -0.7716649048948594


200it [00:00, 774.35it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9387389939293618
Objective PLSRegression_Objective:


  6%|██▎                                   | 499/8000 [00:00<00:06, 1074.49it/s]
63it [00:00, 629.72it/s]

iterations: 499
Number of used features: 26
Used features: ['num-p048-leg.nii.gz-0.500000', 'sum-p048-leg.nii.gz-0.500000', 'mean-p048-leg.nii.gz-0.500000', 'std-p048-leg.nii.gz-0.500000', 'skew-p048-leg.nii.gz-0.500000', 'kurt-p048-leg.nii.gz-0.500000', 'hist-0-p048-leg.nii.gz-0.500000', 'hist-1-p048-leg.nii.gz-0.500000', 'hist-2-p048-leg.nii.gz-0.500000', 'hist-3-p048-leg.nii.gz-0.500000', 'hist-4-p048-leg.nii.gz-0.500000', 'hist-5-p048-leg.nii.gz-0.500000', 'hist-6-p048-leg.nii.gz-0.500000', 'hist-7-p048-leg.nii.gz-0.500000', 'hist-8-p048-leg.nii.gz-0.500000', 'hist-9-p048-leg.nii.gz-0.500000', 'hist-10-p048-leg.nii.gz-0.500000', 'hist-11-p048-leg.nii.gz-0.500000', 'hist-12-p048-leg.nii.gz-0.500000', 'hist-13-p048-leg.nii.gz-0.500000', 'hist-14-p048-leg.nii.gz-0.500000', 'hist-15-p048-leg.nii.gz-0.500000', 'hist-16-p048-leg.nii.gz-0.500000', 'hist-17-p048-leg.nii.gz-0.500000', 'hist-18-p048-leg.nii.gz-0.500000', 'hist-19-p048-leg.nii.gz-0.500000'] 
Score: -0.9142706552629516


200it [00:00, 631.28it/s]

1
1 0.9216316347317524


### loin all features

In [31]:
results_no_fs.append(model_selection(loin_features, loin_target, dataset='loin', type='all', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 529/8000 [00:20<04:47, 25.98it/s]
69it [00:00, 688.00it/s]

iterations: 529
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-loin.nii.gz-0.500000', 'sum-p004-loin.nii.gz-0.500000', 'mean-p004-loin.nii.gz-0.500000', 'std-p004-loin.nii.gz-0.500000', 'skew-p004-loin.nii.gz-0.500000', 'kurt-p004-loin.nii.gz-0.500000', 'hist-0-p004-loin.nii.gz-0.500000', 'hist-1-p004-loin.nii.gz-0.500000', 'hist-2-p004-loin.nii.gz-0.500000', 'hist-3-p004-loin.nii.gz-0.500000', 'hist-4-p004-loin.nii.gz-0.500000', 'hist-5-p004-loin.nii.gz-0.500000', 'hist-6-p004-loin.nii.gz-0.500000', 'hist-7-p004-loin.nii.gz-0.500000', 'hist-8-p004-loin.nii.gz-0.500000', 'hist-9-p004-loin.nii.gz-0.500000', 'hist-10-p004-loin.nii.gz-0.500000', 'hist-11-p004-loin.nii.gz-0.500000', 'hist-12-p004-loin.nii.gz-0.500000', 'hist-13-p004-loin.nii.gz-0.500000', 'hist-14-p004-loin.nii.gz-0.500000', 'hist-15-p004-loin.nii.gz-0.500000', 'hist-16-p004-loin.nii.gz-0.500000', 'hist-17-p004-loin.nii.gz-0.500000', 'hist-18-p004-loin.nii.gz-0.500000', 'hist-19-p004-loin.nii.gz-0.500

200it [00:00, 687.19it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8760693949355225
Objective PLSRegression_Objective:


  7%|██▌                                    | 528/8000 [00:02<00:29, 257.11it/s]
56it [00:00, 551.91it/s]

iterations: 528
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-loin.nii.gz-0.500000', 'sum-p004-loin.nii.gz-0.500000', 'mean-p004-loin.nii.gz-0.500000', 'std-p004-loin.nii.gz-0.500000', 'skew-p004-loin.nii.gz-0.500000', 'kurt-p004-loin.nii.gz-0.500000', 'hist-0-p004-loin.nii.gz-0.500000', 'hist-1-p004-loin.nii.gz-0.500000', 'hist-2-p004-loin.nii.gz-0.500000', 'hist-3-p004-loin.nii.gz-0.500000', 'hist-4-p004-loin.nii.gz-0.500000', 'hist-5-p004-loin.nii.gz-0.500000', 'hist-6-p004-loin.nii.gz-0.500000', 'hist-7-p004-loin.nii.gz-0.500000', 'hist-8-p004-loin.nii.gz-0.500000', 'hist-9-p004-loin.nii.gz-0.500000', 'hist-10-p004-loin.nii.gz-0.500000', 'hist-11-p004-loin.nii.gz-0.500000', 'hist-12-p004-loin.nii.gz-0.500000', 'hist-13-p004-loin.nii.gz-0.500000', 'hist-14-p004-loin.nii.gz-0.500000', 'hist-15-p004-loin.nii.gz-0.500000', 'hist-16-p004-loin.nii.gz-0.500000', 'hist-17-p004-loin.nii.gz-0.500000', 'hist-18-p004-loin.nii.gz-0.500000', 'hist-19-p004-loin.nii.gz-0.500

200it [00:00, 545.68it/s]

1
1 0.8912495563266198


### loin mean mask

In [32]:
results_no_fs.append(model_selection(loin_features[[c for c in loin_features.columns if 'mean_mask' in c or c == 'type']], loin_target, dataset='loin', type='mean_mask', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  6%|██▌                                     | 511/8000 [00:13<03:22, 37.03it/s]
79it [00:00, 781.36it/s]

iterations: 511
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.7102160602999603


200it [00:00, 781.93it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8951313739666346
Objective PLSRegression_Objective:


  6%|██▎                                   | 499/8000 [00:00<00:06, 1088.84it/s]
63it [00:00, 624.51it/s]

iterations: 499
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.8863994421217554


200it [00:00, 621.94it/s]

1
1 0.8934025830429194


### loin individual masks

In [33]:
for m in masks:
    results_no_fs.append(model_selection(loin_features[[c for c in loin_features.columns if m in c or c == 'type']], loin_target, dataset='loin', type=m, disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 534/8000 [00:14<03:17, 37.77it/s]
78it [00:00, 771.80it/s]

iterations: 534
Number of used features: 26
Used features: ['num-p004-loin.nii.gz-0.500000', 'sum-p004-loin.nii.gz-0.500000', 'mean-p004-loin.nii.gz-0.500000', 'std-p004-loin.nii.gz-0.500000', 'skew-p004-loin.nii.gz-0.500000', 'kurt-p004-loin.nii.gz-0.500000', 'hist-0-p004-loin.nii.gz-0.500000', 'hist-1-p004-loin.nii.gz-0.500000', 'hist-2-p004-loin.nii.gz-0.500000', 'hist-3-p004-loin.nii.gz-0.500000', 'hist-4-p004-loin.nii.gz-0.500000', 'hist-5-p004-loin.nii.gz-0.500000', 'hist-6-p004-loin.nii.gz-0.500000', 'hist-7-p004-loin.nii.gz-0.500000', 'hist-8-p004-loin.nii.gz-0.500000', 'hist-9-p004-loin.nii.gz-0.500000', 'hist-10-p004-loin.nii.gz-0.500000', 'hist-11-p004-loin.nii.gz-0.500000', 'hist-12-p004-loin.nii.gz-0.500000', 'hist-13-p004-loin.nii.gz-0.500000', 'hist-14-p004-loin.nii.gz-0.500000', 'hist-15-p004-loin.nii.gz-0.500000', 'hist-16-p004-loin.nii.gz-0.500000', 'hist-17-p004-loin.nii.gz-0.500000', 'hist-18-p004-loin.nii.gz-0.500000', 'hist-19-p004-loin.nii.gz-0.500000'] 
Score: -

200it [00:00, 771.46it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8964188051686747
Objective PLSRegression_Objective:


  6%|██▎                                   | 499/8000 [00:00<00:07, 1053.99it/s]
63it [00:00, 622.55it/s]

iterations: 499
Number of used features: 26
Used features: ['num-p048-loin.nii.gz-0.500000', 'sum-p048-loin.nii.gz-0.500000', 'mean-p048-loin.nii.gz-0.500000', 'std-p048-loin.nii.gz-0.500000', 'skew-p048-loin.nii.gz-0.500000', 'kurt-p048-loin.nii.gz-0.500000', 'hist-0-p048-loin.nii.gz-0.500000', 'hist-1-p048-loin.nii.gz-0.500000', 'hist-2-p048-loin.nii.gz-0.500000', 'hist-3-p048-loin.nii.gz-0.500000', 'hist-4-p048-loin.nii.gz-0.500000', 'hist-5-p048-loin.nii.gz-0.500000', 'hist-6-p048-loin.nii.gz-0.500000', 'hist-7-p048-loin.nii.gz-0.500000', 'hist-8-p048-loin.nii.gz-0.500000', 'hist-9-p048-loin.nii.gz-0.500000', 'hist-10-p048-loin.nii.gz-0.500000', 'hist-11-p048-loin.nii.gz-0.500000', 'hist-12-p048-loin.nii.gz-0.500000', 'hist-13-p048-loin.nii.gz-0.500000', 'hist-14-p048-loin.nii.gz-0.500000', 'hist-15-p048-loin.nii.gz-0.500000', 'hist-16-p048-loin.nii.gz-0.500000', 'hist-17-p048-loin.nii.gz-0.500000', 'hist-18-p048-loin.nii.gz-0.500000', 'hist-19-p048-loin.nii.gz-0.500000'] 
Score: -

200it [00:00, 619.47it/s]

1
1 0.9046528509083862


### belly with all features

In [34]:
results_no_fs.append(model_selection(belly_features, belly_target, dataset='belly', type='all', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 529/8000 [00:20<04:43, 26.32it/s]
71it [00:00, 704.58it/s]

iterations: 529
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-belly.nii.gz-0.500000', 'sum-p004-belly.nii.gz-0.500000', 'mean-p004-belly.nii.gz-0.500000', 'std-p004-belly.nii.gz-0.500000', 'skew-p004-belly.nii.gz-0.500000', 'kurt-p004-belly.nii.gz-0.500000', 'hist-0-p004-belly.nii.gz-0.500000', 'hist-1-p004-belly.nii.gz-0.500000', 'hist-2-p004-belly.nii.gz-0.500000', 'hist-3-p004-belly.nii.gz-0.500000', 'hist-4-p004-belly.nii.gz-0.500000', 'hist-5-p004-belly.nii.gz-0.500000', 'hist-6-p004-belly.nii.gz-0.500000', 'hist-7-p004-belly.nii.gz-0.500000', 'hist-8-p004-belly.nii.gz-0.500000', 'hist-9-p004-belly.nii.gz-0.500000', 'hist-10-p004-belly.nii.gz-0.500000', 'hist-11-p004-belly.nii.gz-0.500000', 'hist-12-p004-belly.nii.gz-0.500000', 'hist-13-p004-belly.nii.gz-0.500000', 'hist-14-p004-belly.nii.gz-0.500000', 'hist-15-p004-belly.nii.gz-0.500000', 'hist-16-p004-belly.nii.gz-0.500000', 'hist-17-p004-belly.nii.gz-0.500000', 'hist-18-p004-belly.nii.gz-0.500000', 'hist-

200it [00:00, 703.00it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8446210845520659
Objective PLSRegression_Objective:


  6%|██▍                                    | 502/8000 [00:01<00:29, 251.45it/s]
70it [00:00, 692.12it/s]

iterations: 502
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-belly.nii.gz-0.500000', 'sum-p004-belly.nii.gz-0.500000', 'mean-p004-belly.nii.gz-0.500000', 'std-p004-belly.nii.gz-0.500000', 'skew-p004-belly.nii.gz-0.500000', 'kurt-p004-belly.nii.gz-0.500000', 'hist-0-p004-belly.nii.gz-0.500000', 'hist-1-p004-belly.nii.gz-0.500000', 'hist-2-p004-belly.nii.gz-0.500000', 'hist-3-p004-belly.nii.gz-0.500000', 'hist-4-p004-belly.nii.gz-0.500000', 'hist-5-p004-belly.nii.gz-0.500000', 'hist-6-p004-belly.nii.gz-0.500000', 'hist-7-p004-belly.nii.gz-0.500000', 'hist-8-p004-belly.nii.gz-0.500000', 'hist-9-p004-belly.nii.gz-0.500000', 'hist-10-p004-belly.nii.gz-0.500000', 'hist-11-p004-belly.nii.gz-0.500000', 'hist-12-p004-belly.nii.gz-0.500000', 'hist-13-p004-belly.nii.gz-0.500000', 'hist-14-p004-belly.nii.gz-0.500000', 'hist-15-p004-belly.nii.gz-0.500000', 'hist-16-p004-belly.nii.gz-0.500000', 'hist-17-p004-belly.nii.gz-0.500000', 'hist-18-p004-belly.nii.gz-0.500000', 'hist-

200it [00:00, 694.04it/s]

1
1 0.8713017012539173


### belly mean mask

In [35]:
results_no_fs.append(model_selection(belly_features[[c for c in belly_features.columns if 'mean_mask' in c or c == 'type']], belly_target, dataset='belly', type='mean_mask', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 527/8000 [00:14<03:23, 36.79it/s]
79it [00:00, 782.00it/s]

iterations: 527
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.7226689224084646


200it [00:00, 786.72it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8646155777499759
Objective PLSRegression_Objective:


  6%|██▍                                   | 502/8000 [00:00<00:06, 1098.18it/s]
79it [00:00, 785.97it/s]

iterations: 502
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.850562098613609


200it [00:00, 784.83it/s]

1
1 0.8544444583222339


### belly individual masks

In [36]:
for m in masks:
    results_no_fs.append(model_selection(belly_features[[c for c in belly_features.columns if m in c or c == 'type']], belly_target, dataset='belly', type=m, disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  8%|███                                     | 603/8000 [00:14<03:02, 40.46it/s]
80it [00:00, 789.70it/s]

iterations: 603
Number of used features: 26
Used features: ['num-p004-belly.nii.gz-0.500000', 'sum-p004-belly.nii.gz-0.500000', 'mean-p004-belly.nii.gz-0.500000', 'std-p004-belly.nii.gz-0.500000', 'skew-p004-belly.nii.gz-0.500000', 'kurt-p004-belly.nii.gz-0.500000', 'hist-0-p004-belly.nii.gz-0.500000', 'hist-1-p004-belly.nii.gz-0.500000', 'hist-2-p004-belly.nii.gz-0.500000', 'hist-3-p004-belly.nii.gz-0.500000', 'hist-4-p004-belly.nii.gz-0.500000', 'hist-5-p004-belly.nii.gz-0.500000', 'hist-6-p004-belly.nii.gz-0.500000', 'hist-7-p004-belly.nii.gz-0.500000', 'hist-8-p004-belly.nii.gz-0.500000', 'hist-9-p004-belly.nii.gz-0.500000', 'hist-10-p004-belly.nii.gz-0.500000', 'hist-11-p004-belly.nii.gz-0.500000', 'hist-12-p004-belly.nii.gz-0.500000', 'hist-13-p004-belly.nii.gz-0.500000', 'hist-14-p004-belly.nii.gz-0.500000', 'hist-15-p004-belly.nii.gz-0.500000', 'hist-16-p004-belly.nii.gz-0.500000', 'hist-17-p004-belly.nii.gz-0.500000', 'hist-18-p004-belly.nii.gz-0.500000', 'hist-19-p004-belly.n

200it [00:00, 787.23it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8167401071024495
Objective PLSRegression_Objective:


  6%|██▍                                   | 507/8000 [00:00<00:07, 1054.24it/s]
67it [00:00, 661.28it/s]

iterations: 507
Number of used features: 26
Used features: ['num-p048-belly.nii.gz-0.500000', 'sum-p048-belly.nii.gz-0.500000', 'mean-p048-belly.nii.gz-0.500000', 'std-p048-belly.nii.gz-0.500000', 'skew-p048-belly.nii.gz-0.500000', 'kurt-p048-belly.nii.gz-0.500000', 'hist-0-p048-belly.nii.gz-0.500000', 'hist-1-p048-belly.nii.gz-0.500000', 'hist-2-p048-belly.nii.gz-0.500000', 'hist-3-p048-belly.nii.gz-0.500000', 'hist-4-p048-belly.nii.gz-0.500000', 'hist-5-p048-belly.nii.gz-0.500000', 'hist-6-p048-belly.nii.gz-0.500000', 'hist-7-p048-belly.nii.gz-0.500000', 'hist-8-p048-belly.nii.gz-0.500000', 'hist-9-p048-belly.nii.gz-0.500000', 'hist-10-p048-belly.nii.gz-0.500000', 'hist-11-p048-belly.nii.gz-0.500000', 'hist-12-p048-belly.nii.gz-0.500000', 'hist-13-p048-belly.nii.gz-0.500000', 'hist-14-p048-belly.nii.gz-0.500000', 'hist-15-p048-belly.nii.gz-0.500000', 'hist-16-p048-belly.nii.gz-0.500000', 'hist-17-p048-belly.nii.gz-0.500000', 'hist-18-p048-belly.nii.gz-0.500000', 'hist-19-p048-belly.n

200it [00:00, 659.04it/s]

1
1 0.825976053803267


### groin all features

In [37]:
results_no_fs.append(model_selection(groin_features, groin_target, dataset='groin', type='all', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


 12%|████▉                                   | 998/8000 [00:27<03:14, 36.02it/s]
55it [00:00, 540.89it/s]

iterations: 998
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-groin.nii.gz-0.500000', 'sum-p004-groin.nii.gz-0.500000', 'mean-p004-groin.nii.gz-0.500000', 'std-p004-groin.nii.gz-0.500000', 'skew-p004-groin.nii.gz-0.500000', 'kurt-p004-groin.nii.gz-0.500000', 'hist-0-p004-groin.nii.gz-0.500000', 'hist-1-p004-groin.nii.gz-0.500000', 'hist-2-p004-groin.nii.gz-0.500000', 'hist-3-p004-groin.nii.gz-0.500000', 'hist-4-p004-groin.nii.gz-0.500000', 'hist-5-p004-groin.nii.gz-0.500000', 'hist-6-p004-groin.nii.gz-0.500000', 'hist-7-p004-groin.nii.gz-0.500000', 'hist-8-p004-groin.nii.gz-0.500000', 'hist-9-p004-groin.nii.gz-0.500000', 'hist-10-p004-groin.nii.gz-0.500000', 'hist-11-p004-groin.nii.gz-0.500000', 'hist-12-p004-groin.nii.gz-0.500000', 'hist-13-p004-groin.nii.gz-0.500000', 'hist-14-p004-groin.nii.gz-0.500000', 'hist-15-p004-groin.nii.gz-0.500000', 'hist-16-p004-groin.nii.gz-0.500000', 'hist-17-p004-groin.nii.gz-0.500000', 'hist-18-p004-groin.nii.gz-0.500000', 'hist-

200it [00:00, 545.55it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.4554177452803624
Objective PLSRegression_Objective:


  6%|██▍                                    | 502/8000 [00:02<00:30, 243.95it/s]
68it [00:00, 678.45it/s]

iterations: 502
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-groin.nii.gz-0.500000', 'sum-p004-groin.nii.gz-0.500000', 'mean-p004-groin.nii.gz-0.500000', 'std-p004-groin.nii.gz-0.500000', 'skew-p004-groin.nii.gz-0.500000', 'kurt-p004-groin.nii.gz-0.500000', 'hist-0-p004-groin.nii.gz-0.500000', 'hist-1-p004-groin.nii.gz-0.500000', 'hist-2-p004-groin.nii.gz-0.500000', 'hist-3-p004-groin.nii.gz-0.500000', 'hist-4-p004-groin.nii.gz-0.500000', 'hist-5-p004-groin.nii.gz-0.500000', 'hist-6-p004-groin.nii.gz-0.500000', 'hist-7-p004-groin.nii.gz-0.500000', 'hist-8-p004-groin.nii.gz-0.500000', 'hist-9-p004-groin.nii.gz-0.500000', 'hist-10-p004-groin.nii.gz-0.500000', 'hist-11-p004-groin.nii.gz-0.500000', 'hist-12-p004-groin.nii.gz-0.500000', 'hist-13-p004-groin.nii.gz-0.500000', 'hist-14-p004-groin.nii.gz-0.500000', 'hist-15-p004-groin.nii.gz-0.500000', 'hist-16-p004-groin.nii.gz-0.500000', 'hist-17-p004-groin.nii.gz-0.500000', 'hist-18-p004-groin.nii.gz-0.500000', 'hist-

200it [00:00, 676.59it/s]

1
1 0.4846585913188459


### groin mean mask

In [38]:
results_no_fs.append(model_selection(groin_features[[c for c in groin_features.columns if 'mean_mask' in c or c == 'type']], groin_target, dataset='groin', type='mean_mask', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  9%|███▍                                    | 686/8000 [00:17<03:05, 39.47it/s]
73it [00:00, 727.93it/s]

iterations: 686
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.36844274889205253


200it [00:00, 728.26it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.5401529105798007
Objective PLSRegression_Objective:


  6%|██▍                                   | 502/8000 [00:00<00:07, 1055.99it/s]
78it [00:00, 773.01it/s]

iterations: 502
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.4988376726883045


200it [00:00, 774.44it/s]

1
1 0.5096841171880576


### groin individual masks

In [39]:
for m in masks:
    results_no_fs.append(model_selection(groin_features[[c for c in groin_features.columns if m in c or c == 'type']], groin_target, dataset='groin', type=m, disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  9%|███▌                                    | 720/8000 [00:17<02:56, 41.32it/s]
73it [00:00, 720.45it/s]

iterations: 720
Number of used features: 26
Used features: ['num-p004-groin.nii.gz-0.500000', 'sum-p004-groin.nii.gz-0.500000', 'mean-p004-groin.nii.gz-0.500000', 'std-p004-groin.nii.gz-0.500000', 'skew-p004-groin.nii.gz-0.500000', 'kurt-p004-groin.nii.gz-0.500000', 'hist-0-p004-groin.nii.gz-0.500000', 'hist-1-p004-groin.nii.gz-0.500000', 'hist-2-p004-groin.nii.gz-0.500000', 'hist-3-p004-groin.nii.gz-0.500000', 'hist-4-p004-groin.nii.gz-0.500000', 'hist-5-p004-groin.nii.gz-0.500000', 'hist-6-p004-groin.nii.gz-0.500000', 'hist-7-p004-groin.nii.gz-0.500000', 'hist-8-p004-groin.nii.gz-0.500000', 'hist-9-p004-groin.nii.gz-0.500000', 'hist-10-p004-groin.nii.gz-0.500000', 'hist-11-p004-groin.nii.gz-0.500000', 'hist-12-p004-groin.nii.gz-0.500000', 'hist-13-p004-groin.nii.gz-0.500000', 'hist-14-p004-groin.nii.gz-0.500000', 'hist-15-p004-groin.nii.gz-0.500000', 'hist-16-p004-groin.nii.gz-0.500000', 'hist-17-p004-groin.nii.gz-0.500000', 'hist-18-p004-groin.nii.gz-0.500000', 'hist-19-p004-groin.n

200it [00:00, 729.88it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.5362764559945106
Objective PLSRegression_Objective:


  6%|██▍                                   | 502/8000 [00:00<00:07, 1036.59it/s]
80it [00:00, 796.05it/s]

iterations: 502
Number of used features: 26
Used features: ['num-p048-groin.nii.gz-0.500000', 'sum-p048-groin.nii.gz-0.500000', 'mean-p048-groin.nii.gz-0.500000', 'std-p048-groin.nii.gz-0.500000', 'skew-p048-groin.nii.gz-0.500000', 'kurt-p048-groin.nii.gz-0.500000', 'hist-0-p048-groin.nii.gz-0.500000', 'hist-1-p048-groin.nii.gz-0.500000', 'hist-2-p048-groin.nii.gz-0.500000', 'hist-3-p048-groin.nii.gz-0.500000', 'hist-4-p048-groin.nii.gz-0.500000', 'hist-5-p048-groin.nii.gz-0.500000', 'hist-6-p048-groin.nii.gz-0.500000', 'hist-7-p048-groin.nii.gz-0.500000', 'hist-8-p048-groin.nii.gz-0.500000', 'hist-9-p048-groin.nii.gz-0.500000', 'hist-10-p048-groin.nii.gz-0.500000', 'hist-11-p048-groin.nii.gz-0.500000', 'hist-12-p048-groin.nii.gz-0.500000', 'hist-13-p048-groin.nii.gz-0.500000', 'hist-14-p048-groin.nii.gz-0.500000', 'hist-15-p048-groin.nii.gz-0.500000', 'hist-16-p048-groin.nii.gz-0.500000', 'hist-17-p048-groin.nii.gz-0.500000', 'hist-18-p048-groin.nii.gz-0.500000', 'hist-19-p048-groin.n

200it [00:00, 792.39it/s]

1
1 0.529547030021888


### shoulder with all features

In [40]:
results_no_fs.append(model_selection(shoulder_features, shoulder_target, dataset='shoulder', type='all', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  9%|███▌                                    | 720/8000 [00:22<03:50, 31.56it/s]
66it [00:00, 657.39it/s]

iterations: 720
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-shoulder.nii.gz-0.500000', 'sum-p004-shoulder.nii.gz-0.500000', 'mean-p004-shoulder.nii.gz-0.500000', 'std-p004-shoulder.nii.gz-0.500000', 'skew-p004-shoulder.nii.gz-0.500000', 'kurt-p004-shoulder.nii.gz-0.500000', 'hist-0-p004-shoulder.nii.gz-0.500000', 'hist-1-p004-shoulder.nii.gz-0.500000', 'hist-2-p004-shoulder.nii.gz-0.500000', 'hist-3-p004-shoulder.nii.gz-0.500000', 'hist-4-p004-shoulder.nii.gz-0.500000', 'hist-5-p004-shoulder.nii.gz-0.500000', 'hist-6-p004-shoulder.nii.gz-0.500000', 'hist-7-p004-shoulder.nii.gz-0.500000', 'hist-8-p004-shoulder.nii.gz-0.500000', 'hist-9-p004-shoulder.nii.gz-0.500000', 'hist-10-p004-shoulder.nii.gz-0.500000', 'hist-11-p004-shoulder.nii.gz-0.500000', 'hist-12-p004-shoulder.nii.gz-0.500000', 'hist-13-p004-shoulder.nii.gz-0.500000', 'hist-14-p004-shoulder.nii.gz-0.500000', 'hist-15-p004-shoulder.nii.gz-0.500000', 'hist-16-p004-shoulder.nii.gz-0.500000', 'hist-17-p004

200it [00:00, 655.30it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8896239735493976
Objective PLSRegression_Objective:


  6%|██▍                                    | 502/8000 [00:02<00:31, 239.28it/s]
67it [00:00, 661.18it/s]

iterations: 502
Number of used features: 157
Used features: ['Unnamed: 0', 'num-p004-shoulder.nii.gz-0.500000', 'sum-p004-shoulder.nii.gz-0.500000', 'mean-p004-shoulder.nii.gz-0.500000', 'std-p004-shoulder.nii.gz-0.500000', 'skew-p004-shoulder.nii.gz-0.500000', 'kurt-p004-shoulder.nii.gz-0.500000', 'hist-0-p004-shoulder.nii.gz-0.500000', 'hist-1-p004-shoulder.nii.gz-0.500000', 'hist-2-p004-shoulder.nii.gz-0.500000', 'hist-3-p004-shoulder.nii.gz-0.500000', 'hist-4-p004-shoulder.nii.gz-0.500000', 'hist-5-p004-shoulder.nii.gz-0.500000', 'hist-6-p004-shoulder.nii.gz-0.500000', 'hist-7-p004-shoulder.nii.gz-0.500000', 'hist-8-p004-shoulder.nii.gz-0.500000', 'hist-9-p004-shoulder.nii.gz-0.500000', 'hist-10-p004-shoulder.nii.gz-0.500000', 'hist-11-p004-shoulder.nii.gz-0.500000', 'hist-12-p004-shoulder.nii.gz-0.500000', 'hist-13-p004-shoulder.nii.gz-0.500000', 'hist-14-p004-shoulder.nii.gz-0.500000', 'hist-15-p004-shoulder.nii.gz-0.500000', 'hist-16-p004-shoulder.nii.gz-0.500000', 'hist-17-p004

200it [00:00, 658.01it/s]

1
1 0.889558249075754


### shoulder mean mask

In [41]:
results_no_fs.append(model_selection(shoulder_features[[c for c in shoulder_features.columns if 'mean_mask' in c or c == 'type']], shoulder_target, dataset='shoulder', type='mean_mask', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 533/8000 [00:15<03:37, 34.29it/s]
75it [00:00, 741.67it/s]

iterations: 533
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.6418814060785882


200it [00:00, 741.54it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8842884917865877
Objective PLSRegression_Objective:


  6%|██▍                                   | 502/8000 [00:00<00:07, 1041.99it/s]
77it [00:00, 766.27it/s]

iterations: 502
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.8735523265189857


200it [00:00, 762.96it/s]

1
1 0.8794783593980505


### shoulder individual masks

In [42]:
for m in masks:
    results_no_fs.append(model_selection(shoulder_features[[c for c in shoulder_features.columns if m in c or c == 'type']], shoulder_target, dataset='shoulder', type=m, disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 545/8000 [00:14<03:25, 36.36it/s]
75it [00:00, 748.82it/s]

iterations: 545
Number of used features: 26
Used features: ['num-p004-shoulder.nii.gz-0.500000', 'sum-p004-shoulder.nii.gz-0.500000', 'mean-p004-shoulder.nii.gz-0.500000', 'std-p004-shoulder.nii.gz-0.500000', 'skew-p004-shoulder.nii.gz-0.500000', 'kurt-p004-shoulder.nii.gz-0.500000', 'hist-0-p004-shoulder.nii.gz-0.500000', 'hist-1-p004-shoulder.nii.gz-0.500000', 'hist-2-p004-shoulder.nii.gz-0.500000', 'hist-3-p004-shoulder.nii.gz-0.500000', 'hist-4-p004-shoulder.nii.gz-0.500000', 'hist-5-p004-shoulder.nii.gz-0.500000', 'hist-6-p004-shoulder.nii.gz-0.500000', 'hist-7-p004-shoulder.nii.gz-0.500000', 'hist-8-p004-shoulder.nii.gz-0.500000', 'hist-9-p004-shoulder.nii.gz-0.500000', 'hist-10-p004-shoulder.nii.gz-0.500000', 'hist-11-p004-shoulder.nii.gz-0.500000', 'hist-12-p004-shoulder.nii.gz-0.500000', 'hist-13-p004-shoulder.nii.gz-0.500000', 'hist-14-p004-shoulder.nii.gz-0.500000', 'hist-15-p004-shoulder.nii.gz-0.500000', 'hist-16-p004-shoulder.nii.gz-0.500000', 'hist-17-p004-shoulder.nii.g

200it [00:00, 757.22it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.8661734689419394
Objective PLSRegression_Objective:


  6%|██▎                                   | 499/8000 [00:00<00:06, 1088.32it/s]
64it [00:00, 632.04it/s]

iterations: 499
Number of used features: 26
Used features: ['num-p048-shoulder.nii.gz-0.500000', 'sum-p048-shoulder.nii.gz-0.500000', 'mean-p048-shoulder.nii.gz-0.500000', 'std-p048-shoulder.nii.gz-0.500000', 'skew-p048-shoulder.nii.gz-0.500000', 'kurt-p048-shoulder.nii.gz-0.500000', 'hist-0-p048-shoulder.nii.gz-0.500000', 'hist-1-p048-shoulder.nii.gz-0.500000', 'hist-2-p048-shoulder.nii.gz-0.500000', 'hist-3-p048-shoulder.nii.gz-0.500000', 'hist-4-p048-shoulder.nii.gz-0.500000', 'hist-5-p048-shoulder.nii.gz-0.500000', 'hist-6-p048-shoulder.nii.gz-0.500000', 'hist-7-p048-shoulder.nii.gz-0.500000', 'hist-8-p048-shoulder.nii.gz-0.500000', 'hist-9-p048-shoulder.nii.gz-0.500000', 'hist-10-p048-shoulder.nii.gz-0.500000', 'hist-11-p048-shoulder.nii.gz-0.500000', 'hist-12-p048-shoulder.nii.gz-0.500000', 'hist-13-p048-shoulder.nii.gz-0.500000', 'hist-14-p048-shoulder.nii.gz-0.500000', 'hist-15-p048-shoulder.nii.gz-0.500000', 'hist-16-p048-shoulder.nii.gz-0.500000', 'hist-17-p048-shoulder.nii.g

200it [00:00, 634.37it/s]

1
1 0.8402397523785139


### body all features

In [43]:
results_no_fs.append(model_selection(body_features, body_target, dataset='body', type='all', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  8%|███                                     | 617/8000 [00:31<06:13, 19.76it/s]
62it [00:00, 612.76it/s]

iterations: 617
Number of used features: 300
Used features: ['num-p004-body.nii.gz-0.500000', 'sum-p004-body.nii.gz-0.500000', 'mean-p004-body.nii.gz-0.500000', 'std-p004-body.nii.gz-0.500000', 'skew-p004-body.nii.gz-0.500000', 'kurt-p004-body.nii.gz-0.500000', 'hist-0-p004-body.nii.gz-0.500000', 'hist-1-p004-body.nii.gz-0.500000', 'hist-2-p004-body.nii.gz-0.500000', 'hist-3-p004-body.nii.gz-0.500000', 'hist-4-p004-body.nii.gz-0.500000', 'hist-5-p004-body.nii.gz-0.500000', 'hist-6-p004-body.nii.gz-0.500000', 'hist-7-p004-body.nii.gz-0.500000', 'hist-8-p004-body.nii.gz-0.500000', 'hist-9-p004-body.nii.gz-0.500000', 'hist-10-p004-body.nii.gz-0.500000', 'hist-11-p004-body.nii.gz-0.500000', 'hist-12-p004-body.nii.gz-0.500000', 'hist-13-p004-body.nii.gz-0.500000', 'hist-14-p004-body.nii.gz-0.500000', 'hist-15-p004-body.nii.gz-0.500000', 'hist-16-p004-body.nii.gz-0.500000', 'hist-17-p004-body.nii.gz-0.500000', 'hist-18-p004-body.nii.gz-0.500000', 'hist-19-p004-body.nii.gz-0.500000', 'hist-20

200it [00:00, 610.64it/s]


...

1
1 0.9724400469953977
Objective PLSRegression_Objective:


  7%|██▋                                     | 544/8000 [00:07<01:43, 72.07it/s]
24it [00:00, 235.06it/s]

iterations: 544
Number of used features: 300
Used features: ['num-p004-body.nii.gz-0.500000', 'sum-p004-body.nii.gz-0.500000', 'mean-p004-body.nii.gz-0.500000', 'std-p004-body.nii.gz-0.500000', 'skew-p004-body.nii.gz-0.500000', 'kurt-p004-body.nii.gz-0.500000', 'hist-0-p004-body.nii.gz-0.500000', 'hist-1-p004-body.nii.gz-0.500000', 'hist-2-p004-body.nii.gz-0.500000', 'hist-3-p004-body.nii.gz-0.500000', 'hist-4-p004-body.nii.gz-0.500000', 'hist-5-p004-body.nii.gz-0.500000', 'hist-6-p004-body.nii.gz-0.500000', 'hist-7-p004-body.nii.gz-0.500000', 'hist-8-p004-body.nii.gz-0.500000', 'hist-9-p004-body.nii.gz-0.500000', 'hist-10-p004-body.nii.gz-0.500000', 'hist-11-p004-body.nii.gz-0.500000', 'hist-12-p004-body.nii.gz-0.500000', 'hist-13-p004-body.nii.gz-0.500000', 'hist-14-p004-body.nii.gz-0.500000', 'hist-15-p004-body.nii.gz-0.500000', 'hist-16-p004-body.nii.gz-0.500000', 'hist-17-p004-body.nii.gz-0.500000', 'hist-18-p004-body.nii.gz-0.500000', 'hist-19-p004-body.nii.gz-0.500000', 'hist-20

200it [00:00, 252.50it/s]

1
1 0.9739205137340873


### body mean mask

In [44]:
results_no_fs.append(model_selection(body_features[[c for c in body_features.columns if 'mean_mask' in c or c == 'type']], body_target, dataset='body', type='mean_mask', disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  7%|██▋                                     | 530/8000 [00:17<04:01, 30.99it/s]
69it [00:00, 683.62it/s]

iterations: 530
Number of used features: 50
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask', 'hist-20-0.500000-mean_mask', 'hist-21-0.500000-mean_mask', 'hist-22-0.500000-mean_mask', 'hist-23-0.500000-mean_mask', 'hist-24-0.500000-mean_mask', 'hist-25-0.500000-mean_mask', 'hist-26-0.

200it [00:00, 676.03it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.973377551161969
Objective PLSRegression_Objective:


  6%|██▍                                    | 500/8000 [00:00<00:13, 571.89it/s]
54it [00:00, 536.08it/s]

iterations: 500
Number of used features: 50
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask', 'hist-20-0.500000-mean_mask', 'hist-21-0.500000-mean_mask', 'hist-22-0.500000-mean_mask', 'hist-23-0.500000-mean_mask', 'hist-24-0.500000-mean_mask', 'hist-25-0.500000-mean_mask', 'hist-26-0.

200it [00:00, 530.03it/s]

1
1 0.9727186295251121


### body individual masks

In [45]:
for m in masks:
    results_no_fs.append(model_selection(body_features[[c for c in body_features.columns if m in c or c == 'type']], body_target, dataset='body', type=m, disable_feature_selection=True))

  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Objective KNNR_Objective:


  6%|██▌                                     | 514/8000 [00:15<03:50, 32.46it/s]
65it [00:00, 649.13it/s]

iterations: 514
Number of used features: 50
Used features: ['num-p004-body.nii.gz-0.500000', 'sum-p004-body.nii.gz-0.500000', 'mean-p004-body.nii.gz-0.500000', 'std-p004-body.nii.gz-0.500000', 'skew-p004-body.nii.gz-0.500000', 'kurt-p004-body.nii.gz-0.500000', 'hist-0-p004-body.nii.gz-0.500000', 'hist-1-p004-body.nii.gz-0.500000', 'hist-2-p004-body.nii.gz-0.500000', 'hist-3-p004-body.nii.gz-0.500000', 'hist-4-p004-body.nii.gz-0.500000', 'hist-5-p004-body.nii.gz-0.500000', 'hist-6-p004-body.nii.gz-0.500000', 'hist-7-p004-body.nii.gz-0.500000', 'hist-8-p004-body.nii.gz-0.500000', 'hist-9-p004-body.nii.gz-0.500000', 'hist-10-p004-body.nii.gz-0.500000', 'hist-11-p004-body.nii.gz-0.500000', 'hist-12-p004-body.nii.gz-0.500000', 'hist-13-p004-body.nii.gz-0.500000', 'hist-14-p004-body.nii.gz-0.500000', 'hist-15-p004-body.nii.gz-0.500000', 'hist-16-p004-body.nii.gz-0.500000', 'hist-17-p004-body.nii.gz-0.500000', 'hist-18-p004-body.nii.gz-0.500000', 'hist-19-p004-body.nii.gz-0.500000', 'hist-20-

200it [00:00, 663.53it/s]
  0%|                                                  | 0/8000 [00:00<?, ?it/s]

...

1
1 0.9635214026804734
Objective PLSRegression_Objective:


  6%|██▍                                    | 500/8000 [00:00<00:12, 587.03it/s]
55it [00:00, 546.55it/s]

iterations: 500
Number of used features: 50
Used features: ['num-p048-body.nii.gz-0.500000', 'sum-p048-body.nii.gz-0.500000', 'mean-p048-body.nii.gz-0.500000', 'std-p048-body.nii.gz-0.500000', 'skew-p048-body.nii.gz-0.500000', 'kurt-p048-body.nii.gz-0.500000', 'hist-0-p048-body.nii.gz-0.500000', 'hist-1-p048-body.nii.gz-0.500000', 'hist-2-p048-body.nii.gz-0.500000', 'hist-3-p048-body.nii.gz-0.500000', 'hist-4-p048-body.nii.gz-0.500000', 'hist-5-p048-body.nii.gz-0.500000', 'hist-6-p048-body.nii.gz-0.500000', 'hist-7-p048-body.nii.gz-0.500000', 'hist-8-p048-body.nii.gz-0.500000', 'hist-9-p048-body.nii.gz-0.500000', 'hist-10-p048-body.nii.gz-0.500000', 'hist-11-p048-body.nii.gz-0.500000', 'hist-12-p048-body.nii.gz-0.500000', 'hist-13-p048-body.nii.gz-0.500000', 'hist-14-p048-body.nii.gz-0.500000', 'hist-15-p048-body.nii.gz-0.500000', 'hist-16-p048-body.nii.gz-0.500000', 'hist-17-p048-body.nii.gz-0.500000', 'hist-18-p048-body.nii.gz-0.500000', 'hist-19-p048-body.nii.gz-0.500000', 'hist-20-

200it [00:00, 543.55it/s]

1
1 0.9660433684008112


### Saving the resulst

In [46]:
results_no_fs= pd.concat(results_no_fs)
results_no_fs.to_csv(os.path.join(path_prefix_results,'results_no_fs.csv'), index=False)
pickle.dump(results_no_fs, open(os.path.join(path_prefix_results,'results_no_fs.pickle'), 'wb'))